In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
bboxes = pd.read_csv('train_ship_segmentations_v2.csv')

In [ ]:
# Display the distribution of NaN values in 'EncodedPixels'
nan_counts = bboxes['EncodedPixels'].isna().sum()
total_images = len(bboxes)

print(f"Number of images with no segmentation mask: {nan_counts}")
print(f"Total number of images: {total_images}")
print(f"Percentage of images without segmentation mask: {nan_counts / total_images * 100:.2f}%")


In [ ]:
# Display the distribution of images with multiple segmentation masks
image_counts = bboxes['ImageId'].value_counts()
multi_mask_images = image_counts[image_counts > 1]

print(f"Number of images with multiple segmentation masks: {len(multi_mask_images)}")
print(f"Percentage of images with multiple segmentation masks: {len(multi_mask_images) / total_images * 100:.2f}%")


In [ ]:
# Display a few examples of images with multiple segmentation masks
sample_multi_mask_images = multi_mask_images.head(3).index

for ImageId in sample_multi_mask_images:
    img_masks = bboxes.loc[bboxes['ImageId'] == ImageId, 'EncodedPixels'].tolist()

    # Display the original image
    img_path = os.path.join(data_dir, ImageId)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Original Image')

    # Display the segmentation masks
    plt.subplot(1, 2, 2)
    for i, mask in enumerate(img_masks):
        if pd.notna(mask) and (len(mask) != 0):
            decoded_mask = rle_decode(mask)
            plt.imshow(decoded_mask, cmap=ListedColormap(['black', 'white']), alpha=0.3, label=f'Mask {i + 1}')
    
    plt.title('Segmentation Masks')
    plt.legend()
    plt.show()